In [ ]:
# importing all libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Conv2D,Flatten,Dense,MaxPool2D,Input
from tensorflow.keras.models import Model,Sequential
import matplotlib.pyplot as plt
import cv2
from PIL import Image
from sklearn.model_selection import train_test_split
import io
import os
from tensorflow.keras.preprocessing.image import img_to_array,load_img
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau
from tensorflow.keras.models import load_model
import random
import imutils

import warnings
warnings.filterwarnings('ignore')

In [ ]:
#load the preprocessed dataset
df = pd.read_csv('./no_zebra_img11.csv')
df['label'] = 'crosswalk'
print(df)

In [ ]:
#making the variables
data=[]
target=[]
filename=[]
images_folder_path=r"./all_images/"

In [ ]:

#collecting the xmin,xmax,ymin,ymax values of the bounding boxes
for index,rows in df.iterrows():
    file,start_x,start_y,end_x,end_y,label=rows["imagePath"],rows["xmin"],rows["ymin"],rows["xmax"],rows["ymax"],rows['label']
    filename.append(file)
    image_path=images_folder_path+file
    image=cv2.imread(image_path)
    h,w,_=image.shape
    assert not isinstance(image,type(None)), 'image not found'
   
    start_x=start_x/w
    start_y=start_y/h
    end_x=end_x/w
    end_y=end_y/h

    #loading image, resizing and converting to array
    image=load_img(image_path,target_size=(224,224))
    image=img_to_array(image)
    #data = all the images
    data.append(image)
    #target = all the labels (boundingboxes)
    target.append((start_x,start_y,end_x,end_y))

In [ ]:
#converting to numpy array
data=np.array(data,dtype="float32")/255.0
targets=np.array(target,dtype="float32")

#splitting the data into train and test
train_images,validation_images,train_targets,validation_targets=train_test_split(data,targets,test_size=0.1,random_state=2022)

In [ ]:
#creating callbacks
checkpoint_filepath = 'best_weights_balanced2.hdf5'

earlyStop = EarlyStopping(monitor='val_accuracy', patience=15, 
                        verbose=1, mode='max',restore_best_weights=True)

checkpoint = ModelCheckpoint(filepath=checkpoint_filepath, 
                                      save_weights_only=False, 
                                      monitor='val_accuracy',
                                      mode='max', 
                                      save_best_only=True)
learningrate = ReduceLROnPlateau(monitor='val_accuracy',
                                          mode='max',
                                          min_delta=0.03, patience=3, 
                                          factor=.5,
                                          min_lr=0.00001, verbose=1)

callbacks = [earlyStop, checkpoint, learningrate]

In [ ]:
#creating the model
model=Sequential()
model.add(Conv2D(64,(3,3),activation="relu",padding="same",input_shape=(224,224,3)))
model.add(Conv2D(64,(3,3),activation="relu",padding="same"))
model.add(MaxPool2D(2,2))
model.add(Conv2D(128,(3,3),activation="relu",padding="same"))
model.add(Conv2D(128,(3,3),activation="relu",padding="same"))
model.add(MaxPool2D(2,2))
model.add(Conv2D(256,(3,3),activation="relu",padding="same"))
model.add(Conv2D(256,(3,3),activation="relu",padding="same"))
model.add(MaxPool2D(2,2))
model.add(Conv2D(512,(3,3),activation="relu",padding="same"))
model.add(Conv2D(512,(3,3),activation="relu",padding="same"))
model.add(Flatten())
model.add(Dense(256,activation="relu"))
model.add(Dense(128,activation="relu"))
model.add(Dense(64,activation="relu"))
model.add(Dense(32,activation="relu"))
model.add(Dense(4,activation="sigmoid"))


opt=Adam(learning_rate=0.0001)
model.compile(loss="mse",optimizer=opt, metrics=['accuracy'])

print(model.summary())

In [ ]:
#use CUDA for GPU if available and train the model
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
with tf.device('/cpu:0'):
    model.fit(train_images,train_targets,validation_data=(validation_images,validation_targets),batch_size=16,epochs=8,callbacks=callbacks,verbose=1)

In [ ]:
#saving the model
model.save('object_detect_balanced2.hdf5')

In [ ]:
#loading the model
model = load_model('best_weights_balanced.hdf5')

In [ ]:
#testing the model
plt.figure(figsize=(12, 24))
test_images_folder_path="/Users/Jorg/Data/crosswalk_data/test_images4/"
# adding images names in dataframe with category.
test_image_path = os.listdir(test_images_folder_path)
test_image_path = [test_images_folder_path + i for i in test_image_path]
# print(test_image_path)
for index,path in enumerate(test_image_path[0:18]):
    test_image=load_img(path,target_size=(224,224))
    test_image=np.array(test_image)/255
    test_image=np.expand_dims(test_image,axis=0)
# make bounding box predictions on the input image
    prediction=model.predict(test_image)[0]
    (startX,startY,endX,endY)=prediction
	# load the input image (in OpenCV format), resize it such that it
	# fits on our screen, and grab its dimensions
    test_img=cv2.imread(path)
    test_img=imutils.resize(test_img,width=600)
    (h,w)=test_img.shape[:2]
    # scale the predicted bounding box coordinates based on the image
	# dimensions
    startX=int(startX*w)
    startY=int(startY*h)
    endX=int(endX*w)
    endY=int(endY*h)
    print(startX,startY,endX,endY)
    # draw a bounding box surrounding the object so we can visualize it
    if (endX-startX>40) and (endY-startY>40):
        cv2.rectangle(test_img,(startX,startY),(endX,endY),(255,0,0),2)
        plt.subplot(6, 3, index+1)
        plt.imshow(test_img)
        
        #outputPath = '/Users/Jorg/Data/crosswalk_data/test_images/'
        #outputPath = os.getcwd()
        #outputFile = path[:-4]+'_detected.jpg'
        #cv2.imwrite(os.path.join(outputPath , outputFile),test_img)
    else:
        cv2.rectangle(test_img,(0,0),(0,0),(255,0,0),2)
        plt.subplot(6, 3, index+1)
        plt.imshow(test_img)
        
plt.tight_layout()
plt.show()
print("[INFO] cleaning up...")
